#### HTML script to introduce table borders for easier analysis

In [1]:
%%HTML
<style type='text/css'>
table.dataframe td, table.dataframe 
th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

### Sections

- 1. Calculate the number of missing data points
- 2. Remove missing data
- 3. Replace missing data
    - 3.1 Replacing missing values with the next value

### Libraries

- Pandas
- NumPy
- IPython

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display

In [3]:
df = pd.read_csv('data/winemag-data-130k-v2.csv', index_col=[0])

# View first three rows of DataFrame
display(df.head(3))

# Produce of DataFrame dimensionality
total_number_cells = np.product(df.shape)

print(f"Number of cells in DataFrame: {total_number_cells}")

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


Number of cells in DataFrame: 1689623


### 1. Calculate the number of missing data points
- number of missing value per column
- total number of missing values in DataFrame
- percentage of DataFrame with a missing value

In [4]:
# Return the number of missing values per column
print(df.isnull().sum())

# Save to variable 
missing_data = df.isnull().sum()

country                     63
description                  0
designation              37465
points                       0
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
dtype: int64


In [5]:
# Return the total sum of missing values in DataFrame
print(missing_data.sum())

# Save to variable 'missingDataTotal'
missing_data_total = missing_data.sum()
print(f"Number of missing data points: {missing_data_total}")

204752
Number of missing data points: 204752


In [6]:
# Percentage of data missing from DataFrame
print(round(missing_data_total / total_number_cells * 100, 2))

# Save to variable 'missingDataTotalPercentage'
missing_percentage = round(missing_data_total / total_number_cells * 100, 2)
print(str(missing_percentage)+'%')

12.12
12.12%


### 2. Remove missing data

In [7]:
# View all columns with at least one missing value
print(missing_data[missing_data > 0])

# Save to variable 'columnsMissingData'
columns_missing_data = missing_data[missing_data > 0]
print(f"\nNumber of columns with missing data: {len(columns_missing_data)}")

country                     63
designation              37465
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
variety                      1
dtype: int64

Number of columns with missing data: 9


In [8]:
# Simple solution: Drop all columns with at least one missing value
df_example = df.dropna(axis='columns').copy()

print(f"Number of columns remaining when .dropna() method called: {df_example.shape[1]}")
display(df_example.head())

Number of columns remaining when .dropna() method called: 4


,description,points,title,winery
0,"Aromas include tropical fruit, broom, brimston...",87,Nicosia 2013 Vulkà Bianco (Etna),Nicosia
1,"This is ripe and fruity, a wine that is smooth...",87,Quinta dos Avidagos 2011 Avidagos Red (Douro),Quinta dos Avidagos
2,"Tart and snappy, the flavors of lime flesh and...",87,Rainstorm 2013 Pinot Gris (Willamette Valley),Rainstorm
3,"Pineapple rind, lemon pith and orange blossom ...",87,St. Julian 2013 Reserve Late Harvest Riesling ...,St. Julian
4,"Much like the regular bottling from 2012, this...",87,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Sweet Cheeks


### 3. Replace missing data

In [9]:
# Concat the first 5 rows & last 5 rows of DataFrame
df = pd.concat([df.head(), df.tail()])

# DataFrame with missing values replaced with a specific value (0)
df_new = df.fillna(0).copy()

# Function
def concatDataFrames(df1, df2):
    '''Concatenates two DataFrames & displays a new DataFrame with respective columns side-by-side'''
    df_concat = pd.concat([df1.set_index(df1.columns[0]), 
                           df2.set_index(df2.columns[0])],
                           axis=1, keys=['DataFrame1', 'DataFrame2'])
    df_concat = df_concat.swaplevel(axis=1)[df1.columns[1:]]
    with pd.option_context('display.max_rows', None, 
                           'display.max_columns', None):   
        display(df_concat.head(), df_concat.tail())
        
# Call function
concatDataFrames(df.head(3), df_new.head(3))

description  \
                                                 DataFrame1   
country                                                       
Italy     Aromas include tropical fruit, broom, brimston...   
Portugal  This is ripe and fruity, a wine that is smooth...   
US        Tart and snappy, the flavors of lime flesh and...   

                                                              designation  \
                                                 DataFrame2    DataFrame1   
country                                                                     
Italy     Aromas include tropical fruit, broom, brimston...  Vulkà Bianco   
Portugal  This is ripe and fruity, a wine that is smooth...      Avidagos   
US        Tart and snappy, the flavors of lime flesh and...           NaN   

                           points                 price             \
            DataFrame2 DataFrame1 DataFrame2 DataFrame1 DataFrame2   
country                                                              
Italy     Vulkà Bianco         87         87        NaN        0.0   
Portugal      Avidagos         87         87       15.0       15.0   
US                   0         87         87       14.0       14.0   

                   province                              region_1  \
                 DataFrame1         DataFrame2         DataFrame1   
country                                                             
Italy     Sicily & Sardinia  Sicily & Sardinia               Etna   
Portugal              Douro              Douro                NaN   
US                   Oregon             Oregon  Willamette Valley   

                                      region_2                     \
                 DataFrame2         DataFrame1         DataFrame2   
country                                                             
Italy                  Etna                NaN                  0   
Portugal                  0                NaN                  0   
US        Willamette Valley  Willamette Valley  Willamette Valley   

            taster_name                taster_twitter_handle                \
             DataFrame1     DataFrame2            DataFrame1    DataFrame2   
country                                                                      
Italy     Kerin O’Keefe  Kerin O’Keefe          @kerinokeefe  @kerinokeefe   
Portugal     Roger Voss     Roger Voss            @vossroger    @vossroger   
US         Paul Gregutt   Paul Gregutt           @paulgwine    @paulgwine    

                                                  title  \
                                             DataFrame1   
country                                                   
Italy                 Nicosia 2013 Vulkà Bianco  (Etna)   
Portugal  Quinta dos Avidagos 2011 Avidagos Red (Douro)   
US        Rainstorm 2013 Pinot Gris (Willamette Valley)   

                                                                variety  \
                                             DataFrame2      DataFrame1   
country                                                                   
Italy                 Nicosia 2013 Vulkà Bianco  (Etna)     White Blend   
Portugal  Quinta dos Avidagos 2011 Avidagos Red (Douro)  Portuguese Red   
US        Rainstorm 2013 Pinot Gris (Willamette Valley)      Pinot Gris   

                                       winery                       
              DataFrame2           DataFrame1           DataFrame2  
country                                                             
Italy        White Blend              Nicosia              Nicosia  
Portugal  Portuguese Red  Quinta dos Avidagos  Quinta dos Avidagos  
US            Pinot Gris            Rainstorm            Rainstorm

description  \
                                                 DataFrame1   
country                                                       
Italy     Aromas include tropical fruit, broom, brimston...   
Portugal  This is ripe and fruity, a wine that is smooth...   
US        Tart and snappy, the flavors of lime flesh and...   

                                                              designation  \
                                                 DataFrame2    DataFrame1   
country                                                                     
Italy     Aromas include tropical fruit, broom, brimston...  Vulkà Bianco   
Portugal  This is ripe and fruity, a wine that is smooth...      Avidagos   
US        Tart and snappy, the flavors of lime flesh and...           NaN   

                           points                 price             \
            DataFrame2 DataFrame1 DataFrame2 DataFrame1 DataFrame2   
country                                                              
Italy     Vulkà Bianco         87         87        NaN        0.0   
Portugal      Avidagos         87         87       15.0       15.0   
US                   0         87         87       14.0       14.0   

                   province                              region_1  \
                 DataFrame1         DataFrame2         DataFrame1   
country                                                             
Italy     Sicily & Sardinia  Sicily & Sardinia               Etna   
Portugal              Douro              Douro                NaN   
US                   Oregon             Oregon  Willamette Valley   

                                      region_2                     \
                 DataFrame2         DataFrame1         DataFrame2   
country                                                             
Italy                  Etna                NaN                  0   
Portugal                  0                NaN                  0   
US        Willamette Valley  Willamette Valley  Willamette Valley   

            taster_name                taster_twitter_handle                \
             DataFrame1     DataFrame2            DataFrame1    DataFrame2   
country                                                                      
Italy     Kerin O’Keefe  Kerin O’Keefe          @kerinokeefe  @kerinokeefe   
Portugal     Roger Voss     Roger Voss            @vossroger    @vossroger   
US         Paul Gregutt   Paul Gregutt           @paulgwine    @paulgwine    

                                                  title  \
                                             DataFrame1   
country                                                   
Italy                 Nicosia 2013 Vulkà Bianco  (Etna)   
Portugal  Quinta dos Avidagos 2011 Avidagos Red (Douro)   
US        Rainstorm 2013 Pinot Gris (Willamette Valley)   

                                                                variety  \
                                             DataFrame2      DataFrame1   
country                                                                   
Italy                 Nicosia 2013 Vulkà Bianco  (Etna)     White Blend   
Portugal  Quinta dos Avidagos 2011 Avidagos Red (Douro)  Portuguese Red   
US        Rainstorm 2013 Pinot Gris (Willamette Valley)      Pinot Gris   

                                       winery                       
              DataFrame2           DataFrame1           DataFrame2  
country                                                             
Italy        White Blend              Nicosia              Nicosia  
Portugal  Portuguese Red  Quinta dos Avidagos  Quinta dos Avidagos  
US            Pinot Gris            Rainstorm            Rainstorm

In [10]:
# DataFrame with missing values replaced with a specific value (forExample)
df_new = df.fillna('EXAMPLE').copy()

# Call function
concatDataFrames(df.head(3), df_new.head(3))

description  \
                                                 DataFrame1   
country                                                       
Italy     Aromas include tropical fruit, broom, brimston...   
Portugal  This is ripe and fruity, a wine that is smooth...   
US        Tart and snappy, the flavors of lime flesh and...   

                                                              designation  \
                                                 DataFrame2    DataFrame1   
country                                                                     
Italy     Aromas include tropical fruit, broom, brimston...  Vulkà Bianco   
Portugal  This is ripe and fruity, a wine that is smooth...      Avidagos   
US        Tart and snappy, the flavors of lime flesh and...           NaN   

                           points                 price             \
            DataFrame2 DataFrame1 DataFrame2 DataFrame1 DataFrame2   
country                                                              
Italy     Vulkà Bianco         87         87        NaN    EXAMPLE   
Portugal      Avidagos         87         87       15.0       15.0   
US             EXAMPLE         87         87       14.0       14.0   

                   province                              region_1  \
                 DataFrame1         DataFrame2         DataFrame1   
country                                                             
Italy     Sicily & Sardinia  Sicily & Sardinia               Etna   
Portugal              Douro              Douro                NaN   
US                   Oregon             Oregon  Willamette Valley   

                                      region_2                     \
                 DataFrame2         DataFrame1         DataFrame2   
country                                                             
Italy                  Etna                NaN            EXAMPLE   
Portugal            EXAMPLE                NaN            EXAMPLE   
US        Willamette Valley  Willamette Valley  Willamette Valley   

            taster_name                taster_twitter_handle                \
             DataFrame1     DataFrame2            DataFrame1    DataFrame2   
country                                                                      
Italy     Kerin O’Keefe  Kerin O’Keefe          @kerinokeefe  @kerinokeefe   
Portugal     Roger Voss     Roger Voss            @vossroger    @vossroger   
US         Paul Gregutt   Paul Gregutt           @paulgwine    @paulgwine    

                                                  title  \
                                             DataFrame1   
country                                                   
Italy                 Nicosia 2013 Vulkà Bianco  (Etna)   
Portugal  Quinta dos Avidagos 2011 Avidagos Red (Douro)   
US        Rainstorm 2013 Pinot Gris (Willamette Valley)   

                                                                variety  \
                                             DataFrame2      DataFrame1   
country                                                                   
Italy                 Nicosia 2013 Vulkà Bianco  (Etna)     White Blend   
Portugal  Quinta dos Avidagos 2011 Avidagos Red (Douro)  Portuguese Red   
US        Rainstorm 2013 Pinot Gris (Willamette Valley)      Pinot Gris   

                                       winery                       
              DataFrame2           DataFrame1           DataFrame2  
country                                                             
Italy        White Blend              Nicosia              Nicosia  
Portugal  Portuguese Red  Quinta dos Avidagos  Quinta dos Avidagos  
US            Pinot Gris            Rainstorm            Rainstorm

description  \
                                                 DataFrame1   
country                                                       
Italy     Aromas include tropical fruit, broom, brimston...   
Portugal  This is ripe and fruity, a wine that is smooth...   
US        Tart and snappy, the flavors of lime flesh and...   

                                                              designation  \
                                                 DataFrame2    DataFrame1   
country                                                                     
Italy     Aromas include tropical fruit, broom, brimston...  Vulkà Bianco   
Portugal  This is ripe and fruity, a wine that is smooth...      Avidagos   
US        Tart and snappy, the flavors of lime flesh and...           NaN   

                           points                 price             \
            DataFrame2 DataFrame1 DataFrame2 DataFrame1 DataFrame2   
country                                                              
Italy     Vulkà Bianco         87         87        NaN    EXAMPLE   
Portugal      Avidagos         87         87       15.0       15.0   
US             EXAMPLE         87         87       14.0       14.0   

                   province                              region_1  \
                 DataFrame1         DataFrame2         DataFrame1   
country                                                             
Italy     Sicily & Sardinia  Sicily & Sardinia               Etna   
Portugal              Douro              Douro                NaN   
US                   Oregon             Oregon  Willamette Valley   

                                      region_2                     \
                 DataFrame2         DataFrame1         DataFrame2   
country                                                             
Italy                  Etna                NaN            EXAMPLE   
Portugal            EXAMPLE                NaN            EXAMPLE   
US        Willamette Valley  Willamette Valley  Willamette Valley   

            taster_name                taster_twitter_handle                \
             DataFrame1     DataFrame2            DataFrame1    DataFrame2   
country                                                                      
Italy     Kerin O’Keefe  Kerin O’Keefe          @kerinokeefe  @kerinokeefe   
Portugal     Roger Voss     Roger Voss            @vossroger    @vossroger   
US         Paul Gregutt   Paul Gregutt           @paulgwine    @paulgwine    

                                                  title  \
                                             DataFrame1   
country                                                   
Italy                 Nicosia 2013 Vulkà Bianco  (Etna)   
Portugal  Quinta dos Avidagos 2011 Avidagos Red (Douro)   
US        Rainstorm 2013 Pinot Gris (Willamette Valley)   

                                                                variety  \
                                             DataFrame2      DataFrame1   
country                                                                   
Italy                 Nicosia 2013 Vulkà Bianco  (Etna)     White Blend   
Portugal  Quinta dos Avidagos 2011 Avidagos Red (Douro)  Portuguese Red   
US        Rainstorm 2013 Pinot Gris (Willamette Valley)      Pinot Gris   

                                       winery                       
              DataFrame2           DataFrame1           DataFrame2  
country                                                             
Italy        White Blend              Nicosia              Nicosia  
Portugal  Portuguese Red  Quinta dos Avidagos  Quinta dos Avidagos  
US            Pinot Gris            Rainstorm            Rainstorm

#### 3.1 Replacing missing values with the next value

In [11]:
# DataFrame with missing values replaced with with the next value (fillna(0) to catch remaining)
df_new = df.fillna(method='bfill', axis=0).fillna(0)

# Call function
concatDataFrames(df.head(4), df_new.head(4))

description  \
                                                 DataFrame1   
country                                                       
Italy     Aromas include tropical fruit, broom, brimston...   
Portugal  This is ripe and fruity, a wine that is smooth...   
US        Tart and snappy, the flavors of lime flesh and...   
US        Pineapple rind, lemon pith and orange blossom ...   

                                                             \
                                                 DataFrame2   
country                                                       
Italy     Aromas include tropical fruit, broom, brimston...   
Portugal  This is ripe and fruity, a wine that is smooth...   
US        Tart and snappy, the flavors of lime flesh and...   
US        Pineapple rind, lemon pith and orange blossom ...   

                   designation                           points             \
                    DataFrame1            DataFrame2 DataFrame1 DataFrame2   
country                                                                      
Italy             Vulkà Bianco          Vulkà Bianco         87         87   
Portugal              Avidagos              Avidagos         87         87   
US                         NaN  Reserve Late Harvest         87         87   
US        Reserve Late Harvest  Reserve Late Harvest         87         87   

              price                      province                     \
         DataFrame1 DataFrame2         DataFrame1         DataFrame2   
country                                                                
Italy           NaN       15.0  Sicily & Sardinia  Sicily & Sardinia   
Portugal       15.0       15.0              Douro              Douro   
US             14.0       14.0             Oregon             Oregon   
US             13.0       13.0           Michigan           Michigan   

                     region_1                                region_2  \
                   DataFrame1           DataFrame2         DataFrame1   
country                                                                 
Italy                    Etna                 Etna                NaN   
Portugal                  NaN    Willamette Valley                NaN   
US          Willamette Valley    Willamette Valley  Willamette Valley   
US        Lake Michigan Shore  Lake Michigan Shore                NaN   

                                    taster_name                      \
                 DataFrame2          DataFrame1          DataFrame2   
country                                                               
Italy     Willamette Valley       Kerin O’Keefe       Kerin O’Keefe   
Portugal  Willamette Valley          Roger Voss          Roger Voss   
US        Willamette Valley        Paul Gregutt        Paul Gregutt   
US        Willamette Valley  Alexander Peartree  Alexander Peartree   

         taster_twitter_handle                \
                    DataFrame1    DataFrame2   
country                                        
Italy             @kerinokeefe  @kerinokeefe   
Portugal            @vossroger    @vossroger   
US                 @paulgwine    @paulgwine    
US                         NaN   @paulgwine    

                                                      title  \
                                                 DataFrame1   
country                                                       
Italy                     Nicosia 2013 Vulkà Bianco  (Etna)   
Portugal      Quinta dos Avidagos 2011 Avidagos Red (Douro)   
US            Rainstorm 2013 Pinot Gris (Willamette Valley)   
US        St. Julian 2013 Reserve Late Harvest Riesling ...   

                                                                    variety  \
                                                 DataFrame2      DataFrame1   
country                                                                       
Italy                     Nicosia 2013 Vulkà Bianco  (Etna)     White Blend   
Portuga

description  \
                                                 DataFrame1   
country                                                       
Italy     Aromas include tropical fruit, broom, brimston...   
Portugal  This is ripe and fruity, a wine that is smooth...   
US        Tart and snappy, the flavors of lime flesh and...   
US        Pineapple rind, lemon pith and orange blossom ...   

                                                             \
                                                 DataFrame2   
country                                                       
Italy     Aromas include tropical fruit, broom, brimston...   
Portugal  This is ripe and fruity, a wine that is smooth...   
US        Tart and snappy, the flavors of lime flesh and...   
US        Pineapple rind, lemon pith and orange blossom ...   

                   designation                           points             \
                    DataFrame1            DataFrame2 DataFrame1 DataFrame2   
country                                                                      
Italy             Vulkà Bianco          Vulkà Bianco         87         87   
Portugal              Avidagos              Avidagos         87         87   
US                         NaN  Reserve Late Harvest         87         87   
US        Reserve Late Harvest  Reserve Late Harvest         87         87   

              price                      province                     \
         DataFrame1 DataFrame2         DataFrame1         DataFrame2   
country                                                                
Italy           NaN       15.0  Sicily & Sardinia  Sicily & Sardinia   
Portugal       15.0       15.0              Douro              Douro   
US             14.0       14.0             Oregon             Oregon   
US             13.0       13.0           Michigan           Michigan   

                     region_1                                region_2  \
                   DataFrame1           DataFrame2         DataFrame1   
country                                                                 
Italy                    Etna                 Etna                NaN   
Portugal                  NaN    Willamette Valley                NaN   
US          Willamette Valley    Willamette Valley  Willamette Valley   
US        Lake Michigan Shore  Lake Michigan Shore                NaN   

                                    taster_name                      \
                 DataFrame2          DataFrame1          DataFrame2   
country                                                               
Italy     Willamette Valley       Kerin O’Keefe       Kerin O’Keefe   
Portugal  Willamette Valley          Roger Voss          Roger Voss   
US        Willamette Valley        Paul Gregutt        Paul Gregutt   
US        Willamette Valley  Alexander Peartree  Alexander Peartree   

         taster_twitter_handle                \
                    DataFrame1    DataFrame2   
country                                        
Italy             @kerinokeefe  @kerinokeefe   
Portugal            @vossroger    @vossroger   
US                 @paulgwine    @paulgwine    
US                         NaN   @paulgwine    

                                                      title  \
                                                 DataFrame1   
country                                                       
Italy                     Nicosia 2013 Vulkà Bianco  (Etna)   
Portugal      Quinta dos Avidagos 2011 Avidagos Red (Douro)   
US            Rainstorm 2013 Pinot Gris (Willamette Valley)   
US        St. Julian 2013 Reserve Late Harvest Riesling ...   

                                                                    variety  \
                                                 DataFrame2      DataFrame1   
country                                                                       
Italy                     Nicosia 2013 Vulkà Bianco  (Etna)     White Blend   
Portuga